In [1]:
import geoengine as ge

from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import threading
import csv

from IPython.display import display, Code

In [2]:
ge.initialize("https://respect.app.geoengine.io/api", admin_token="515c0fde-588a-469c-a82e-c18b8d3aa528")

In [3]:
session = ge.get_session()
session

Server:              https://respect.app.geoengine.io/api
Session Id:          f5a11a59-2fa1-43e7-8abc-83448b54ccf7
Session valid until: 2023-02-02T14:23:28.913Z

In [4]:
def cloud_free_band_agg_workflow_definition(utm_zone: str, band: str = "B04", step: int = 1, granularity: str = "months"):
    return {
        'type': 'Raster',
        'operator': {
            'type': 'TemporalRasterAggregation',
            'params': {
                'aggregation': {
                    'type': 'mean',
                    'ignoreNoData': True
                },
                'window': {
                    'granularity': granularity,
                    'step': step
                }
            },
            'sources': {
                'raster': {
                    'type': 'Expression',
                    'params': {
                        'expression': '''
                            if ((B > 3 && B < 8) || B == 11) { A } else { NODATA }
                        ''',
                        'outputType': 'U16',
                        'outputMeasurement': None,
                        'mapNoData': False,
                    },
                    'sources': {
                        'a': {
                            'type': 'GdalSource',
                            'params': {
                                'data': {
                                    'type': 'external',
                                    'providerId': '5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5',
                                    'layerId': f'{utm_zone}:{band}'
                                }
                            }
                        },
                        'b': {
                            'type': 'GdalSource',
                            'params': {
                                'data': {
                                    'type': 'external',
                                    'providerId': '5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5',
                                    'layerId': f'{utm_zone}:SCL'
                                }
                            }
                        }
                    }
                }
            }
        }
    }

cloud_free_band_agg_workflow_definition('UTM17S', 'B04', step=1, granularity="months")

{'type': 'Raster',
 'operator': {'type': 'TemporalRasterAggregation',
  'params': {'aggregation': {'type': 'mean', 'ignoreNoData': True},
   'window': {'granularity': 'months', 'step': 1}},
  'sources': {'raster': {'type': 'Expression',
    'params': {'expression': '\n                            if ((B > 3 && B < 8) || B == 11) { A } else { NODATA }\n                        ',
     'outputType': 'U16',
     'outputMeasurement': None,
     'mapNoData': False},
    'sources': {'a': {'type': 'GdalSource',
      'params': {'data': {'type': 'external',
        'providerId': '5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5',
        'layerId': 'UTM17S:B04'}}},
     'b': {'type': 'GdalSource',
      'params': {'data': {'type': 'external',
        'providerId': '5779494c-f3a2-48b3-8a2d-5fbba8c5b6c5',
        'layerId': 'UTM17S:SCL'}}}}}}}}

In [5]:
step = 1
granularity = "months"

workflows = {
    'UTM17S_B04': ge.register_workflow(cloud_free_band_agg_workflow_definition('UTM17S', 'B04', step=step, granularity=granularity)),
    'UTM17S_B08': ge.register_workflow(cloud_free_band_agg_workflow_definition('UTM17S', 'B08', step=step, granularity=granularity)),
}

workflows

{'UTM17S_B04': 3906c66a-2b94-5639-a8cf-357a0e607c42,
 'UTM17S_B08': 2ec74e30-ecaf-5c97-96a4-e2615c07222b}

In [6]:
for workflow in workflows.values():
    display(workflow.get_result_descriptor())

Data type:         U16
Spatial Reference: EPSG:32717
Measurement:       unitless

Data type:         U16
Spatial Reference: EPSG:32717
Measurement:       unitless

In [15]:
time = ge.TimeInterval(datetime.strptime(f'2019-05-01Z', "%Y-%m-%d%z"), datetime.strptime(f'2019-12-01Z', "%Y-%m-%d%z"))
bounds = ge.SpatialPartition2D(594175, 9510945, 733476, 9600019)
pixel_size = 10
resolution = ge.SpatialResolution(pixel_size, pixel_size)

display((bounds.xmax-bounds.xmin)/pixel_size, (bounds.ymax-bounds.ymin)/pixel_size)

13930.1

8907.4

In [16]:
crop_bounds = ge.SpatialPartition2D(650000, 9520000, 730000, 9600000)
crop_pixel_size = 10
crop_resolution = ge.SpatialResolution(crop_pixel_size, crop_pixel_size)

display((crop_bounds.xmax-crop_bounds.xmin)/crop_pixel_size, (crop_bounds.ymax-crop_bounds.ymin)/crop_pixel_size)

8000.0

8000.0

In [17]:
query_rectangle = ge.api.RasterQueryRectangle(
    spatialBounds=crop_bounds.to_api_dict(), timeInterval=time.to_api_dict(), spatialResolution=crop_resolution.to_api_dict() )

query_rectangle


{'spatialBounds': {'upperLeftCoordinate': {'x': 650000, 'y': 9600000},
  'lowerRightCoordinate': {'x': 730000, 'y': 9520000}},
 'timeInterval': {'start': '2019-05-01T00:00:00.000+00:00',
  'end': '2019-12-01T00:00:00.000+00:00'},
 'spatialResolution': {'x': 10, 'y': 10}}

In [18]:
tasks = {}

for workflow_name in workflows:
    workflow = workflows[workflow_name]
    name = workflow_name

    name_with_time = f"{name} - {time.time_str}"
    task = workflow.save_as_dataset(query_rectangle=query_rectangle, name=name_with_time, description=f"Sentinel-2 {name} {time.time_str} as {step} {granularity} steps", timeout=3600*6)

    print("Scheduled", workflow_name)
    
    tasks[workflow_name] = task


Scheduled UTM17S_B04
Scheduled UTM17S_B08


In [20]:
for workflow_name in workflows:
    task = tasks[workflow_name]
    task_status = task.get_status()

    if task_status.status.name == 'RUNNING':
        there_are_tasks_running = True

    print(workflow_name)
    print(task_status)


UTM17S_B04
status=running, pct_complete=0.00%, time_estimate=? (± ?), info=None
UTM17S_B08
status=running, pct_complete=0.00%, time_estimate=? (± ?), info=None


In [21]:
for workflow_name in workflows:
    task = tasks[workflow_name]
    task_status = task.get_status()
    
    if task_status.status.name != 'COMPLETED':
        continue
    
    print(workflow_name)
    
    stored_dataset = ge.StoredDataset.from_response(task_status.info)
    stored_dataset
        
    display_workflow_definition = {
        'type': 'Raster',
        'operator': {
            'type': 'GdalSource',
            'params': {
                'data': {
                    'type': 'internal',
                    'datasetId': str(stored_dataset.dataset_id),
                }
            }
        }
    }

    display_workflow = ge.register_workflow(display_workflow_definition)

    query_rectangle = ge.QueryRectangle(
                spatial_bounds=ge.BoundingBox2D(crop_bounds.xmin, crop_bounds.ymin, crop_bounds.xmax, crop_bounds.ymax),
                time_interval=ge.TimeInterval(time.start),
                resolution=crop_resolution,
                srs="EPSG:32717"
            )

    display(
        display_workflow.wms_get_map_as_image(
            query_rectangle,
            colorizer=ge.Colorizer.linear_with_mpl_cmap('magma', min_max=(-60000,60000))
        )
    )